In [32]:
using CSV

In [34]:
words_nltk = CSV.read("../data/words_nltk.csv");

In [35]:
words_nltk = Array(words_nltk[:2]);

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[35]:1
└ @ Core In[35]:1


In [36]:
words_nltk

236736-element Array{String,1}:
 "A"        
 "a"        
 "aa"       
 "aal"      
 "aalii"    
 "aam"      
 "Aani"     
 "aardvark" 
 "aardwolf" 
 "Aaron"    
 "Aaronic"  
 "Aaronical"
 "Aaronite" 
 ⋮          
 "word"     
 "work"     
 "worm"     
 "wound"    
 "writing"  
 "wrong"    
 "year"     
 "yellow"   
 "yes"      
 "yesterday"
 "you"      
 "young"    

In [37]:
function edit_distance(X::String,Y::String)
    len_x = length(X)
    len_y = length(Y)
    D = zeros(Int, len_x+1,len_y+1)

    @inbounds for i in 1:(len_x+1)
        for j in 1:(len_y+1)
            if i==1
                D[i,j] = j
            elseif j==1
                D[i,j] = i
            elseif X[i-1] == Y[j-1]
                D[i,j] = D[i-1,j-1]
            else
                D[i,j] = 1+min(D[i,j-1], D[i-1,j], D[i-1,j-1])
            end
        end
    end
    return D[len_x,len_y]
end

edit_distance (generic function with 1 method)

In [38]:
edit_distance("lik","cat")

3

In [41]:
using BenchmarkTools

In [42]:
@benchmark edit_distance("exponential", "polynomial")

BenchmarkTools.Trial: 
  memory estimate:  1.14 KiB
  allocs estimate:  1
  --------------
  minimum time:     387.776 ns (0.00% GC)
  median time:      403.776 ns (0.00% GC)
  mean time:        450.889 ns (4.93% GC)
  maximum time:     6.260 μs (87.73% GC)
  --------------
  samples:          10000
  evals/sample:     201

In [43]:
function compute_distances(mistake,words) 
    cy_distances = []
    for word in words
        #ed = edit_distance(mistake, word)
        push!(cy_distances,edit_distance(mistake, word))
    end
    return cy_distances
end

compute_distances (generic function with 1 method)

In [44]:
mistake = "drauing"
@time distances = compute_distances(mistake,words_nltk); closest_word = words_nltk[argmin(distances)]

  0.132923 seconds (243.33 k allocations: 181.485 MiB, 20.07% gc time)


"draine"

In [45]:
@benchmark compute_distances(mistake,words_nltk)

BenchmarkTools.Trial: 
  memory estimate:  181.19 MiB
  allocs estimate:  236754
  --------------
  minimum time:     84.817 ms (5.48% GC)
  median time:      92.001 ms (9.94% GC)
  mean time:        92.580 ms (9.69% GC)
  maximum time:     104.442 ms (14.45% GC)
  --------------
  samples:          55
  evals/sample:     1

In [46]:
function compute_distances2(mistake,words) 
    cy_distances = zeros(Int64,length(words))
    for (i,word) in enumerate(words)
        cy_distances[i] = edit_distance(mistake, word)
    end
    return cy_distances
end

compute_distances2 (generic function with 1 method)

In [47]:
using BenchmarkTools

In [48]:
mistake = "drauing"
@time distances = compute_distances2(mistake,words_nltk); closest_word = words_nltk[argmin(distances)]

  0.102357 seconds (249.78 k allocations: 180.627 MiB, 11.86% gc time)


"draine"

In [49]:
@benchmark compute_distances2($mistake, $words_nltk)

BenchmarkTools.Trial: 
  memory estimate:  180.00 MiB
  allocs estimate:  236738
  --------------
  minimum time:     82.382 ms (4.15% GC)
  median time:      91.012 ms (9.37% GC)
  mean time:        92.056 ms (9.30% GC)
  maximum time:     116.768 ms (12.84% GC)
  --------------
  samples:          55
  evals/sample:     1

In [50]:
@time cy_distances = zeros(Int64,length(words_nltk));

  0.002577 seconds (50 allocations: 1.809 MiB)


In [52]:
function compute_distances3(mistake,words) 
    cy_distances = zeros(Int64,length(words))
    for i in 1:length(words)
        cy_distances[i] = edit_distance(mistake, words[i])
    end
    return cy_distances
end

compute_distances3 (generic function with 1 method)

In [53]:
@benchmark compute_distances3($mistake, $words_nltk)

BenchmarkTools.Trial: 
  memory estimate:  180.00 MiB
  allocs estimate:  236738
  --------------
  minimum time:     81.567 ms (4.07% GC)
  median time:      91.518 ms (9.14% GC)
  mean time:        91.744 ms (9.35% GC)
  maximum time:     109.734 ms (9.85% GC)
  --------------
  samples:          55
  evals/sample:     1

### Multithreading version

In [54]:
Base.Threads.nthreads()

8

In [55]:
function compute_distances4(mistake,words) 
     cy_distances = zeros(Int64,length(words))
     Threads.@threads for i in 1:length(words)
        cy_distances[i] = edit_distance(mistake, words[i])
    end
    return cy_distances
end

compute_distances4 (generic function with 1 method)

In [56]:
mistake = "drauing"
@time distances = compute_distances4(mistake,words_nltk); closest_word =  words_nltk[argmin(distances)]

  0.082818 seconds (288.83 k allocations: 182.673 MiB, 25.39% gc time)


"draine"

In [59]:
using StaticArrays

In [60]:
function edit_distance_opt(X::String,Y::String)
    len_x = length(X)
    len_y = length(Y)
    D = StaticArrays.zeros(Int, len_x+1, len_y+1)

    @inbounds for i in 1:(len_x+1)
        for j in 1:(len_y+1)
            if i==1
                D[i,j] = j
            elseif j==1
                D[i,j] = i
            elseif X[i-1] == Y[j-1]
                D[i,j] = D[i-1,j-1]
            else
                D[i,j] = 1+min(D[i,j-1], D[i-1,j], D[i-1,j-1])
            end
        end
    end
    return D[len_x,len_y]
end

edit_distance_opt (generic function with 1 method)

In [61]:
@benchmark edit_distance_opt("lik","cat")

BenchmarkTools.Trial: 
  memory estimate:  208 bytes
  allocs estimate:  1
  --------------
  minimum time:     70.334 ns (0.00% GC)
  median time:      72.413 ns (0.00% GC)
  mean time:        79.702 ns (5.24% GC)
  maximum time:     1.392 μs (88.86% GC)
  --------------
  samples:          10000
  evals/sample:     976

In [62]:
function compute_distances_static(mistake,words) 
     cy_distances = zeros(Int64,length(words))
     Threads.@threads for i in 1:length(words)
        cy_distances[i] = edit_distance_opt(mistake, words[i])
    end
    return cy_distances
end

compute_distances_static (generic function with 1 method)

In [64]:
mistake = "drauing"
@time distances = compute_distances_static(mistake,words_nltk);

  0.028281 seconds (236.80 k allocations: 180.006 MiB)


In [65]:
@benchmark distances = compute_distances_static(mistake, words_nltk)

BenchmarkTools.Trial: 
  memory estimate:  180.01 MiB
  allocs estimate:  236796
  --------------
  minimum time:     19.449 ms (0.00% GC)
  median time:      54.227 ms (57.61% GC)
  mean time:        41.103 ms (45.32% GC)
  maximum time:     62.321 ms (53.86% GC)
  --------------
  samples:          122
  evals/sample:     1

### String distances


In [72]:
using StringDistances

In [73]:
evaluate(Levenshtein(), "New York", "New Yorks")

1

In [74]:
function compute_distances_stringdistances(mistake,words) 
     measure = Levenshtein()
     cy_distances = zeros(Int64,length(words))
     Threads.@threads for i in 1:length(words)
        cy_distances[i] = evaluate(measure, mistake, words[i])
    end
    return cy_distances
end

compute_distances_stringdistances (generic function with 1 method)

In [ ]:
@benchmark distances = compute_distances_stringdistances(mistake, words_nltk)